In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageOps
import numpy as np
import glob

In [3]:
# 이미지 리사이징 및 검정색 배경으로 채우는 함수
def resize_image_fill_black(image_path, target_size):
    image = Image.open(image_path)
    new_image = ImageOps.fit(image, target_size, method=0, bleed=0.0, centering=(0.5, 0.5), fillcolor=(0, 0, 0))
    resized_image = np.array(new_image)
    return resized_image

In [4]:
# 데이터셋 경로 설정
train_directory = 'D:/note_deep_learning/train_note'
test_directory = 'D:/note_deep_learning/test_note'

# 데이터 증강 및 리사이징 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

# 훈련 및 테스트 데이터셋 로드
target_size = (64, 64)  # 원하는 크기로 변경
batch_size = 32

In [5]:
training_set = train_datagen.flow_from_directory(
    train_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 26458 images belonging to 9 classes.


In [6]:
test_set = test_datagen.flow_from_directory(
    test_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 965 images belonging to 9 classes.


In [7]:
print("Training Set 클래스 이름: ", training_set.class_indices)
print("Test Set 클래스 이름: ", test_set.class_indices)

Training Set 클래스 이름:  {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8}
Test Set 클래스 이름:  {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8}


In [8]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(9, activation='softmax'))
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [9]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 30)#epochs=훈련횟수, 결과 값 안나오면 숫자 늘려가기


Epoch 1/30
827/827 [==============================] - 82s 99ms/step - loss: 0.9443 - accuracy: 0.6593 - val_loss: 0.4469 - val_accuracy: 0.8694
Epoch 2/30
827/827 [==============================] - 82s 100ms/step - loss: 0.3367 - accuracy: 0.8881 - val_loss: 0.3289 - val_accuracy: 0.9140
Epoch 3/30
827/827 [==============================] - 82s 99ms/step - loss: 0.2010 - accuracy: 0.9351 - val_loss: 0.4062 - val_accuracy: 0.8984
Epoch 4/30
827/827 [==============================] - 80s 96ms/step - loss: 0.1405 - accuracy: 0.9538 - val_loss: 0.2398 - val_accuracy: 0.9420
Epoch 5/30
827/827 [==============================] - 82s 99ms/step - loss: 0.1155 - accuracy: 0.9622 - val_loss: 0.2353 - val_accuracy: 0.9326
Epoch 6/30
827/827 [==============================] - 80s 97ms/step - loss: 0.0982 - accuracy: 0.9679 - val_loss: 0.2294 - val_accuracy: 0.9337
Epoch 7/30
827/827 [==============================] - 79s 96ms/step - loss: 0.0830 - accuracy: 0.9726 - val_loss: 0.3711 - val_accuracy

In [10]:
from tensorflow.keras.preprocessing import image

# 이미지 파일이 있는 디렉토리 경로
image_directory = 'D:/note_deep_learning/test/'

# 이미지 파일 목록 가져오기
image_files = glob.glob(image_directory + '*.jpg')  # 필요한 이미지 확장자에 따라 수정

# 클래스 인덱스와 클래스 이름 매핑
class_indices = training_set.class_indices
class_names = {v: k for k, v in class_indices.items()}

# 각 이미지에 대한 예측 수행
for image_file in image_files:
    # 이미지 로드 및 전처리
    test_image = image.load_img(image_file, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    # 예측 수행
    result = cnn.predict(test_image)

    # 예측 결과 확인
    predicted_class_index = np.argmax(result)
    predicted_class_name = class_names[predicted_class_index]

    print(f"Image: {image_file}, Predicted Class: {predicted_class_name}")

1/1 [==============================] - 0s 116ms/step
Image: D:/note_deep_learning/test\2.jpg, Predicted Class: 2
1/1 [==============================] - 0s 27ms/step
Image: D:/note_deep_learning/test\2_1.jpg, Predicted Class: 2
1/1 [==============================] - 0s 26ms/step
Image: D:/note_deep_learning/test\2_2.jpg, Predicted Class: 2
1/1 [==============================] - 0s 27ms/step
Image: D:/note_deep_learning/test\2_3.jpg, Predicted Class: 2
1/1 [==============================] - 0s 27ms/step
Image: D:/note_deep_learning/test\2_4.jpg, Predicted Class: 2
1/1 [==============================] - 0s 28ms/step
Image: D:/note_deep_learning/test\3.jpg, Predicted Class: 3
1/1 [==============================] - 0s 29ms/step
Image: D:/note_deep_learning/test\4.jpg, Predicted Class: 9
1/1 [==============================] - 0s 27ms/step
Image: D:/note_deep_learning/test\4_1.jpg, Predicted Class: 4
1/1 [==============================] - 0s 24ms/step
Image: D:/note_deep_learning/test\4_2.jpg

In [13]:
from tensorflow.keras.models import load_model
import json  # json 모듈을 import합니다.

# 모델을 저장할 때 클래스 정보와 함께 저장합니다.
cnn.save('DrumNote_CNN_Model.keras')
with open('class_indices.json', 'w') as f:
    json.dump(class_indices, f)
